In [6]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = 'c:/Users/balob/Downloads/DATA'

In [3]:
# Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
'''
df_tadig = pd.read_sql_query(sql_srt, cnxn)
#df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
display(df_tadig.head())

cnxn.close()

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4.0
1,Afghanistan,Areeba/MTN,AFGAR,5.0
2,Afghanistan,Etisalat,AFG55,6.0
3,Afghanistan,Etisalat,AFGEA,6.0
4,Afghanistan,Roshan/TDCA,AFGK9,7.0


In [28]:
# Загружаем новые тарифы Partner и P4 и Allowed list

df_partner = pd.read_csv(join(load_folder,'Partner_1_1_2019.csv'),sep=';')
print('Тарифы Partner:')
display(df_partner.head())

df_p4 = pd.read_csv(join(load_folder,'P4_1_1_2019.csv'),sep=';')
print('Тарифы P4:')
display(df_p4.head())

df_al=pd.read_csv(join(load_folder,'DMI.dmi_allowed_list_export_Tue_Dec_18_2018.csv'),usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_CODE','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al = df_al[df_al.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al = df_al.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al = pd.merge(df_al,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',on='TADIG_CODE')
#df_al.drop('TADIG_CODE',inplace=True,axis=1)
#print('Allowed list:')
#display(df_al.head())

#Объединяем тарифы с TADIG и Allowed list

df_partner_net_id = pd.merge(df_partner,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
#print('Тарифы Partner с добавлением network_id:')
#display(df_partner_net_id.head())

df_p4_net_id = pd.merge(df_p4,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
#print('Тарифы P4 с добавлением network_id:')
#display(df_p4_net_id.head())

df_s1_s2_temp = pd.merge(df_partner_net_id[['MOC','MTC','SMS','MB','NETWORK_ID']],
                    df_p4_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','P4_CAMEL','P4_LTE']],
                    how='outer',on=['NETWORK_ID'],suffixes=('_S1','_S2')).drop_duplicates()
print('Тарифы Partner и P4:')
display(df_s1_s2_temp.head())

df_s1_s2 = pd.merge(df_s1_s2_temp,df_tadig,how='left',on='NETWORK_ID')
print('Тарифы Partner и P4 с TADIG:')
display(df_s1_s2.head())

df_s1_s2_sponsor = pd.merge(df_s1_s2,df_al,how='outer',on='NETWORK_ID')
print('Тарифы Partner и P4 с привязкой к Allowed list:')
display(df_s1_s2_sponsor.head())

Тарифы Partner:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB
0,ALBVF,27602.0,Vodafone AL,Albania,VF,0.0439,0.0,0.0062,0.0030
1,ANDMA,21303.0,Andorra Telecom SAU,Andorra,NaN,0.2000,0.0,0.0300,0.2000
2,AIACW,36584.0,AIACW AI,Anguilla,C&W Lime,0.1606,0.0,0.0128,0.0257
3,ATGCW,34492.0,ATGCW AG,Antigua And Barbuda,C&W Lime,0.1500,0.0,0.0120,0.0240
4,ARGTM,72207.0,TELE MOV AR,Argentina,Telefonica LATAM,0.0728,0.0,0.0240,0.0180


Тарифы P4:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,P4_CAMEL,P4_LTE
0,Albania,ALBtelecom sh.a.,ALBEM,0.390,NaN,0.040,0.039,yes,yes
1,Anguilla,Cable and Wireless (Anguilla) Limited,AIACW,0.800,NaN,0.080,0.080,yes,no
2,Antigua,Cable & Wireless Antigua and Barbuda Limited,ATGCW,0.800,NaN,0.080,0.080,yes,no
3,Armenia,Armenia Telephone Company (ArmenTel),ARM01,0.585,NaN,0.015,0.004,yes,no
4,Australia,SingTel Optus Pty Limited,AUSOP,0.055,NaN,0.008,0.008,yes,yes


Тарифы Partner и P4:


,MOC_S1,MTC_S1,SMS_S1,MB_S1,NETWORK_ID,MOC_S2,MTC_S2,SMS_S2,MB_S2,P4_CAMEL,P4_LTE
0,0.0439,0.0,0.0062,0.0030,12.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.2000,0.0,0.0300,0.2000,17.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0.1606,0.0,0.0128,0.0257,20.0,0.8,NaN,0.08,0.08,yes,no
3,0.1500,0.0,0.0120,0.0240,23.0,0.8,NaN,0.08,0.08,yes,no
4,0.0728,0.0,0.0240,0.0180,27.0,NaN,NaN,NaN,NaN,NaN,NaN


Тарифы Partner и P4 с TADIG:


,MOC_S1,MTC_S1,SMS_S1,MB_S1,NETWORK_ID,MOC_S2,MTC_S2,SMS_S2,MB_S2,P4_CAMEL,P4_LTE,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE
0,0.0439,0.0,0.0062,0.0030,12.0,NaN,NaN,NaN,NaN,NaN,NaN,Albania,Vodafone,ALBVF
1,0.2000,0.0,0.0300,0.2000,17.0,NaN,NaN,NaN,NaN,NaN,NaN,Andorra,Andorra Telecom S A U,ANDMA
2,0.1606,0.0,0.0128,0.0257,20.0,0.8,NaN,0.08,0.08,yes,no,Anguilla,Cable and Wireless,AIACW
3,0.1500,0.0,0.0120,0.0240,23.0,0.8,NaN,0.08,0.08,yes,no,Antigua and Barbuda,C & W,ATGCW
4,0.0728,0.0,0.0240,0.0180,27.0,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,Movistar/Telefonica,ARG01


Тарифы Partner и P4 с привязкой к Allowed list:


,MOC_S1,MTC_S1,SMS_S1,MB_S1,NETWORK_ID,MOC_S2,MTC_S2,SMS_S2,MB_S2,P4_CAMEL,P4_LTE,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE_x,SPONSOR,TADIG_CODE_y
0,0.0439,0.0,0.0062,0.0030,12.0,NaN,NaN,NaN,NaN,NaN,NaN,Albania,Vodafone,ALBVF,S1,ALBVF
1,0.2000,0.0,0.0300,0.2000,17.0,NaN,NaN,NaN,NaN,NaN,NaN,Andorra,Andorra Telecom S A U,ANDMA,NaN,NaN
2,0.1606,0.0,0.0128,0.0257,20.0,0.8,NaN,0.08,0.08,yes,no,Anguilla,Cable and Wireless,AIACW,S1,AIACW
3,0.1500,0.0,0.0120,0.0240,23.0,0.8,NaN,0.08,0.08,yes,no,Antigua and Barbuda,C & W,ATGCW,S1,ATGCW
4,0.0728,0.0,0.0240,0.0180,27.0,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,Movistar/Telefonica,ARG01,S1,ARG01


In [19]:
# Результирующий файл

#f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S1','MOC_S2','MTC_S1','MTC_S2','SMS_S1','SMS_S2','MB_S1','MB_S2','SPONSOR']
f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE',
          'MOC_S1','MTC_S1','SMS_S1','MB_S1',
          'MOC_S2','MTC_S2','SMS_S2','MB_S2','SPONSOR']
df_s1_s2_sponsor = df_s1_s2_sponsor[f_list]
#df_s1_s2_sponsor.to_csv(join(load_folder,'Partner_P4_Compare_1_1_2019.csv'),index=False)
df_s1_s2_sponsor.head(10)

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,SPONSOR
0,Albania,Vodafone,ALBVF,0.0439,0.0,0.0062,0.0030,NaN,NaN,NaN,NaN,S1
1,Andorra,Andorra Telecom S A U,ANDMA,0.2000,0.0,0.0300,0.2000,NaN,NaN,NaN,NaN,NaN
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,0.800,NaN,0.080,0.080,S1
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,0.800,NaN,0.080,0.080,S1
4,Argentina,Movistar/Telefonica,ARG01,0.0728,0.0,0.0240,0.0180,NaN,NaN,NaN,NaN,S1
5,Argentina,Movistar/Telefonica,ARGTM,0.0728,0.0,0.0240,0.0180,NaN,NaN,NaN,NaN,S1
6,Armenia,ArmenTel,ARM01,0.0250,0.0,0.0500,0.0050,0.585,NaN,0.015,0.004,S1
7,Armenia,UCOM LLC,ARMOR,0.1255,0.0,0.0251,0.0376,NaN,NaN,NaN,NaN,S1
8,Armenia,Vivacell,ARM05,0.2400,0.0,0.0150,0.0090,NaN,NaN,NaN,NaN,NaN
9,Australia,Singtel Optus,AUSOP,0.0300,0.0,0.0100,0.0030,0.055,NaN,0.008,0.008,S1


In [43]:
df_partner_net_id.head()

,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
0,ALBVF,27602.0,Vodafone AL,Albania,VF,0.0439,0.0,0.0062,0.0030,Albania,Vodafone,ALBVF,12.0
1,ANDMA,21303.0,Andorra Telecom SAU,Andorra,NaN,0.2000,0.0,0.0300,0.2000,Andorra,Andorra Telecom S A U,ANDMA,17.0
2,AIACW,36584.0,AIACW AI,Anguilla,C&W Lime,0.1606,0.0,0.0128,0.0257,Anguilla,Cable and Wireless,AIACW,20.0
3,ATGCW,34492.0,ATGCW AG,Antigua And Barbuda,C&W Lime,0.1500,0.0,0.0120,0.0240,Antigua and Barbuda,C & W,ATGCW,23.0
4,ARGTM,72207.0,TELE MOV AR,Argentina,Telefonica LATAM,0.0728,0.0,0.0240,0.0180,Argentina,Movistar/Telefonica,ARGTM,27.0


In [51]:
df = df_p4_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
df[df.values > 1]

,MOC,MTC,SMS,MB
NETWORK_ID,,,,
412.0,2,0,1,1


In [52]:
df_p4_net_id[df_p4_net_id.NETWORK_ID==412]

,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,P4_CAMEL,P4_LTE,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
86,India,Bharti Airtel Limited,INDBL,0.40,NaN,0.14,0.04,no,no,India,Bharti Airtel Limited (Delhi),INDBL,412.0
87,India,Bharti Airtel Limited,INDJH,0.40,NaN,0.14,0.04,no,no,India,Bharti Airtel Limited (Delhi),INDJH,412.0
88,India,Bharti Airtel Limited,INDSC,0.40,NaN,0.14,0.04,no,no,India,Bharti Airtel Limited (Delhi),INDSC,412.0
89,India,Bharti Airtel Limited,INDAT,0.41,NaN,0.14,0.04,no,no,India,Bharti Airtel Limited (Delhi),INDAT,412.0
90,India,Bharti Airtel Limited,INDA3,0.41,NaN,0.14,0.04,no,no,India,Bharti Airtel Limited (Delhi),INDA3,412.0
91,India,Bharti Airtel Limited,INDA5,0.41,NaN,0.14,0.04,no,no,India,Bharti Airtel Limited (Delhi),INDA5,412.0
92,India,Bharti Airtel Limited,INDJB,0.41,NaN,0.14,0.04,no,no,India,Bharti Airtel Limited (Delhi),INDJB,412.0
93,India,Bharti Airtel Limited,INDA7,0.41,NaN,0.14,0.04,no,no,India,Bharti Airtel Limited (Delhi),INDA7,412.0
94,India,Bharti Airtel Limited,INDMT,0.41,NaN,0.14,0.04,no,no,India,Bharti Airtel Limited (Delhi),INDMT,412.0
95,India,Bharti Airtel Limited,INDA8,0.41,NaN,0.14,0.04,no,no,India,Bharti Airtel Limited (Delhi),INDA8,412.0
